In [1]:
import torch
import torch.nn
from Preprocessing import Patchify, Embeddings

device = "cuda" if torch.cuda.is_available() else "cpu"

rand_tensor = torch.rand((10, 4, 32, 32)).to(device=device, dtype=torch.float32)
# vae = VAE.VAE("mse")
# vae.encode(rand_tensor).shape

patchify = Patchify.Patchify(4, 512)
tokens = patchify(rand_tensor).to(device=device, dtype=torch.float32)

pos = Embeddings.positional_embedding(64, 512)
print(pos.shape)
tokens = tokens + pos.unsqueeze(0)
print(tokens.shape)
      
# Build conditioning tokens (B, 2, d)
num_timesteps = 1000
y_dim = 1000

# Timesteps: [B] longs (0..num_timesteps-1)
t = torch.randint(low=0, high=num_timesteps, size=(10,), device=device, dtype=torch.float32)

# Labels: [B] longs (0..y_dim-1)
y = torch.randint(low=0, high=y_dim, size=(10,), device=device, dtype=torch.long)

InContextCondition = Embeddings.InContextConditioning(512, 256, 1000)
cond_tokens = InContextCondition(t, y).to(tokens.dtype).to(tokens.device)
print(cond_tokens.shape)



torch.Size([64, 512])
torch.Size([10, 64, 512])
torch.Size([10, 2, 512])


In [4]:
import my_models
blocks = my_models.DiTBlockList(d=512, blocks=10, type="In_Context")

In [5]:
seq = blocks(tokens, cond_tokens)

In [7]:

img_tokens = seq[:, :64, :]
img_tokens.shape

torch.Size([10, 64, 512])

In [8]:
unpatchify = Patchify.Unpatchify(d=512, p=4)
out = unpatchify(img_tokens)
out.shape

torch.Size([10, 8, 32, 32])